In [1]:
from __future__ import division, unicode_literals
import numpy as np
from bqplot import Axis, Bars, Figure, Lines, LogScale, Scatter, pyplot
from collections import namedtuple

from colour import Structure, dot_vector, tstack

  File "/Users/kelsolaar/Documents/Development/colour-science/colour/colour/__init__.py", line 294, in __getattr__
    return super(colour, self).__getattr__(attribute)
  File "/Users/kelsolaar/Documents/Development/colour-science/colour/colour/utilities/deprecation.py", line 240, in __getattr__
    warning(str(change))
/Users/kelsolaar/Documents/Development/colour-science/colour/colour/utilities/verbose.py:209: ColourWarning: "colour.Structure" object access is deprecated and will change to "colour.utilities.Structure" in a future release.
  warn(*args, **kwargs)
  File "/Users/kelsolaar/Documents/Development/colour-science/colour/colour/__init__.py", line 294, in __getattr__
    return super(colour, self).__getattr__(attribute)
  File "/Users/kelsolaar/Documents/Development/colour-science/colour/colour/utilities/deprecation.py", line 240, in __getattr__
    warning(str(change))
/Users/kelsolaar/Documents/Development/colour-science/colour/colour/utilities/verbose.py:209: ColourWarning

In [2]:
HALF_MIN = 2 ** -14

M = np.array([
        [0.5, -1.0, 0.5],
        [-1.0, 1.0, 0.0],
        [0.5,  0.5, 0.0]])

SplinePoint = namedtuple('SplinePoint', ('x', 'y'))

SegmentedSplineParams = namedtuple(
    'SegmentedSplineParams',
    ('coefs_low',   # coefs for B-spline between minPoint and midPoint (units of log luminance)
     'coefs_high',  # coefs for B-spline between midPoint and maxPoint (units of log luminance)
     'min_point',   # linear extension below this
     'mid_point',
     'max_point',   # linear extension above this
     'slope_low',   # log-log slope of low linear extension
     'slope_high')) # log-log slope of high linear extension

RRT_PARAMS = SegmentedSplineParams(
    np.array([-4.0000000000, -4.0000000000, -3.1573765773, -0.4852499958, 1.8477324706, 1.8477324706]),
    np.array([-0.7185482425, 2.0810307172, 3.6681241237, 4.0000000000, 4.0000000000, 4.0000000000]),
    SplinePoint(0.18 * 2 ** -15, 0.0001),
    SplinePoint(0.18, 4.8),
    SplinePoint(0.18 * 2 ** 18, 10000),
    0,
    0)

ODT_48nits = SegmentedSplineParams(
    np.array([-1.6989700043, -1.6989700043, -1.4779000000, -1.2291000000, -0.8648000000, -0.4480000000, 0.0051800000, 0.4511080334, 0.9113744414, 0.9113744414]),
    np.array([0.5154386965, 0.8470437783, 1.1358000000, 1.3802000000, 1.5197000000, 1.5985000000, 1.6467000000, 1.6746091357, 1.6878733390, 1.6878733390]),
    SplinePoint(0.18 * 2 ** -6.5, 0.02),
    SplinePoint(0.18, 4.8),
    SplinePoint(0.18 * 2 ** 6.5, 48.0),
    0,
    0.04)


def segmented_spline_fwd(x, C, knots_low, knots_high):
    N_KNOTS_LOW = knots_low
    N_KNOTS_HIGH = knots_high
    
    log_x = np.log10(np.maximum(x, HALF_MIN))
    log_y = np.zeros(log_x.shape)

    mask = log_x <= np.log10(C.min_point.x)
    log_y[mask] = (
        log_x[mask] * C.slope_low + (np.log10(C.min_point.y) - C.slope_low * np.log10(C.min_point.x)))

    mask = np.logical_and(log_x > np.log10(C.min_point.x), log_x < np.log10(C.mid_point.x))
    knot_coord = ((N_KNOTS_LOW - 1) * 
                  (log_x[mask] - np.log10(C.min_point[0])) / (np.log10(C.mid_point[0]) - np.log10(C.min_point[0])))
    j = knot_coord.astype(np.int_)
    t = knot_coord - j
    cf = tstack((C.coefs_low[j], C.coefs_low[j + 1], C.coefs_low[j + 2]))
    monomials = tstack((t * t, t, np.ones(t.shape)))
    log_y[mask] = np.einsum('ij, ij->i', monomials, dot_vector(M, cf))

    mask = np.logical_and(log_x >= np.log10(C.mid_point.x), log_x < np.log10(C.max_point.x))
    knot_coord = ((N_KNOTS_HIGH - 1) * 
                  (log_x[mask] - np.log10(C.mid_point.x)) / (np.log10(C.max_point.x) - np.log10(C.mid_point.x)))
    j = knot_coord.astype(np.int_)
    t = knot_coord - j
    cf = tstack((C.coefs_high[j], C.coefs_high[j + 1], C.coefs_high[j + 2]))
    monomials = tstack((t * t, t, np.ones(t.shape)))
    log_y[mask] = np.einsum('ij, ij->i', monomials, dot_vector(M, cf))

    mask = log_x >= np.log10(C.max_point.x) 
    log_y[mask] = (
        log_x[mask] * C.slope_high + (np.log10(C.max_point.y) - C.slope_high * np.log10(C.max_point.x)))
    
    return 10 ** log_y


def segmented_spline_c5_fwd(x, C=RRT_PARAMS):
    return segmented_spline_fwd(x, C=C, knots_low=4, knots_high=4)


def segmented_spline_c9_fwd(x, C=RRT_PARAMS):
    return segmented_spline_fwd(x, C=C, knots_low=8, knots_high=8)

In [3]:
S_P = ODT_48nits
S_FUNCTION = segmented_spline_c9_fwd

MINIMUM_REPR_NUMBER = S_P.min_point.x
MAXIMUM_REPR_NUMBER = S_P.max_point.x

DEFAULT_SAMPLE_COUNT = 1024

SAMPLES_LOG = np.logspace(
    np.log10(MINIMUM_REPR_NUMBER), 
    np.log10(MAXIMUM_REPR_NUMBER), 
    DEFAULT_SAMPLE_COUNT)
SAMPLES = np.linspace(
    SAMPLES_LOG[0], 
    SAMPLES_LOG[-1], 
    DEFAULT_SAMPLE_COUNT / 4)

SPLINE_SCALE_X = LogScale()
SPLINE_SCALE_Y = LogScale()

SPLINE_AXIS_X = Axis(scale=SPLINE_SCALE_X, grid_lines='solid', label='X')
SPLINE_AXIS_Y = Axis(scale=SPLINE_SCALE_Y, grid_lines='solid', label='Y', orientation='vertical')

SPLINE_LINE = Lines(
    x=SAMPLES_LOG, 
    y=S_FUNCTION(SAMPLES_LOG, S_P), 
    scales={'x': SPLINE_SCALE_X, 'y': SPLINE_SCALE_Y})

SPLINE_SCATTER = Scatter(
    x=[S_P.min_point.x, 
       S_P.mid_point.x, 
       S_P.max_point.x],
    y=[S_P.min_point.y, 
       S_P.mid_point.y, 
       S_P.max_point.y],
    scales={'x': SPLINE_SCALE_X, 'y': SPLINE_SCALE_Y},
    enable_move=True,
    update_on_move=True)

Figure(marks=[SPLINE_LINE, SPLINE_SCATTER], 
       axes=[SPLINE_AXIS_X, SPLINE_AXIS_Y], 
       title='Segmented Spline Interactive')

  File "<ipython-input-3-883d1f33c265>", line 16, in <module>
    DEFAULT_SAMPLE_COUNT / 4)
  File "/usr/local/anaconda3/envs/colour-2.7/lib/python2.7/site-packages/numpy/core/function_base.py", line 101, in linspace
    num = _index_deprecate(num)
/usr/local/anaconda3/envs/colour-2.7/lib/python2.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.
  app.launch_new_instance()


RmlndXJlKGF4ZXM9W0F4aXMobGFiZWw9dSdYJywgc2NhbGU9TG9nU2NhbGUoKSksIEF4aXMobGFiZWw9dSdZJywgb3JpZW50YXRpb249dSd2ZXJ0aWNhbCcsIHNjYWxlPUxvZ1NjYWxlKCkpXSzigKY=


In [4]:
def on_spline_scatter_drag(name, value):
    s_p = SegmentedSplineParams(
        S_P.coefs_low,
        S_P.coefs_high,
        SplinePoint(SPLINE_SCATTER.x[0], SPLINE_SCATTER.y[0]),
        SplinePoint(SPLINE_SCATTER.x[1], SPLINE_SCATTER.y[1]),
        SplinePoint(SPLINE_SCATTER.x[2], SPLINE_SCATTER.y[2]),
        S_P.slope_low,
        S_P.slope_high)

    SPLINE_LINE.y = S_FUNCTION(SAMPLES_LOG, s_p)

SPLINE_SCATTER.on_drag_start(on_spline_scatter_drag)
SPLINE_SCATTER.on_drag(on_spline_scatter_drag)
SPLINE_SCATTER.on_drag_end(on_spline_scatter_drag)